In [42]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import statsmodels.api as sm
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from lightgbm import LGBMRegressor
from sklearn.ensemble import VotingRegressor # 앙상블

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [2]:
month_crime = pd.read_csv('crime_seoul_all.csv', encoding='cp949', index_col=0)
weather = pd.read_csv('s_weather.csv', encoding='cp949')
pop = pd.read_csv('crime_pop_all.csv', encoding='cp949')

In [57]:
weather = pd.merge(month_crime, weather, on='일시')

In [60]:
# 독립 변수와 종속 변수 설정
X = weather[['강수량(mm)', '일최다강수량(mm)', '평균기온(℃)', '평균최고기온(℃)', '최고기온(℃)',
       '평균최저기온(℃)', '최저기온(℃)', '평균풍속(m/s)', '최대풍속(m/s)', '최대순간풍속(m/s)',
       '평균습도(%rh)', '최저습도(%rh)']] # 독립 변수
y = weather['월_범죄건수'] # 종속 변수

# 훈련 데이터와 테스트 데이터로 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [37]:
# 랜덤 포레스트 회귀 모델 생성
model_rf = RandomForestRegressor(n_estimators=100, random_state=42)

# 모델 학습
model_rf.fit(X_train, y_train)

# 테스트 데이터로 예측
y_pred_rf = model_rf.predict(X_test)

# 평가 지표 계산 (예: 평균 제곱 오차)
mse_rf = mean_squared_error(y_test, y_pred_rf)
print("Mean Squared Error:", mse_rf)

# MAE 계산
mae_rf = mean_absolute_error(y_test, y_pred_rf)
print("Mean Absolute Error:", mae_rf)

# RMSE 계산
rmse_rf = np.sqrt(mean_squared_error(y_test, y_pred_rf))
print("Root Mean Squared Error:", rmse_rf)

Mean Squared Error: 966405.2198727273
Mean Absolute Error: 752.3981818181818
Root Mean Squared Error: 983.0591131120891


In [38]:
# 서포트벡터머신 회귀 모델 생성
model_svm = SVR()

# 모델 학습
model_svm.fit(X_train, y_train)

# 테스트 데이터로 예측
y_pred_svm = model_svm.predict(X_test)

# 평가 지표 계산 (예: 평균 제곱 오차)
mse_svm = mean_squared_error(y_test, y_pred_svm)
print("Mean Squared Error:", mse_svm)

# MAE 계산
mae_svm = mean_absolute_error(y_test, y_pred_svm)
print("Mean Absolute Error:", mae_svm)

# RMSE 계산
rmse_svm = np.sqrt(mean_squared_error(y_test, y_pred_svm))
print("Root Mean Squared Error:", rmse_svm)

Mean Squared Error: 1185931.0654800946
Mean Absolute Error: 825.8025048307103
Root Mean Squared Error: 1089.0046214227443


In [40]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 의사결정트리 회귀 모델 생성
model_dt = DecisionTreeRegressor(random_state=42)

# 모델 학습
model_dt.fit(X_train, y_train)

# 테스트 데이터로 예측
y_pred_dt = model_dt.predict(X_test)

# 평가 지표 계산 (예: 평균 제곱 오차)
mse_dt = mean_squared_error(y_test, y_pred_dt)
print("Mean Squared Error:", mse_dt)

# MAE 계산
mae_dt = mean_absolute_error(y_test, y_pred_dt)
print("Mean Absolute Error:", mae_dt)

# RMSE 계산
rmse_dt = np.sqrt(mean_squared_error(y_test, y_pred_dt))
print("Root Mean Squared Error:", rmse_dt)


Mean Squared Error: 1816338.2727272727
Mean Absolute Error: 1091.5454545454545
Root Mean Squared Error: 1347.7159466027226


### 예측 모델 정확도 비교

In [49]:
# 각 모델의 평가 지표 결과
results = {
    'Model': ['RF', 'SVM', 'DT'],
    'MSE': [round(mse_rf, 2), round(mse_svm, 2), round(mse_dt, 2)],
    'MAE': [round(mae_rf, 2), round(mae_svm, 2), round(mae_dt, 2)],
    'RMSE': [round(rmse_rf, 2), round(rmse_svm, 2), round(rmse_dt, 2)]
}



# 데이터프레임 생성
results_df = pd.DataFrame(results)
results_df.to_csv('model_result.csv', encoding='cp949', index=False)
results_df

,Model,MSE,MAE,RMSE
0,RF,966405.22,752.40,983.06
1,SVM,1185931.07,825.80,1089.00
2,DT,1816338.27,1091.55,1347.72


### 이상치 처리 및 앙상블 기법

In [50]:
# 랜덤 포레스트 회귀 모델 생성
model_rf = RandomForestRegressor(n_estimators=100, random_state=42)

# 모델 학습
model_rf.fit(X_train, y_train)

# 테스트 데이터로 예측
y_pred_rf = model_rf.predict(X_test)

# 평가 지표 계산 (예: 평균 제곱 오차)
mse_rf = mean_squared_error(y_test, y_pred_rf)
print("Mean Squared Error:", mse_rf)

# MAE 계산
mae_rf = mean_absolute_error(y_test, y_pred_rf)
print("Mean Absolute Error:", mae_rf)

# RMSE 계산
rmse_rf = np.sqrt(mean_squared_error(y_test, y_pred_rf))
print("Root Mean Squared Error:", rmse_rf)

Mean Squared Error: 966405.2198727273
Mean Absolute Error: 752.3981818181818
Root Mean Squared Error: 983.0591131120891


In [59]:
def remove_outliers(df, column, threshold=3):
    mean = df[column].mean()
    std = df[column].std()
    z_scores = (df[column] - mean) / std
    outliers = df[z_scores.abs() > threshold]  # 임계값을 초과하는 Z-score를 가진 데이터는 이상치로 간주
    df = df.drop(outliers.index)  # 이상치 제거
    return df

# 이상치 식별 및 처리
threshold = 3  # 임계값 설정
column = '월_범죄건수'  # 대상 칼럼 설정
weather_no_outliers = remove_outliers(weather, column, threshold)

# 처리 후 데이터 확인
print("이상치 처리 전 데이터 개수:", len(weather))
print("이상치 처리 후 데이터 개수:", len(weather_no_outliers))


이상치 처리 전 데이터 개수: 36
이상치 처리 후 데이터 개수: 36
